<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install eli5
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107747 sha256=7f64f1dc4ca7b78a5495ce09b3ba9d01e445530b50486579d39f9253c5d102a5
  Stored in directory: /root/.cache/pip/wheels/7b/26/a5/8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import scipy
import numpy as np

#error =  cannot import itemfreq from scipyl.stats
# below code solved the issue 
def monkeypath_itemfreq(sampler_indices):
   return zip(*np.unique(sampler_indices, return_counts=True))
scipy.stats.itemfreq=monkeypath_itemfreq

import eli5
from eli5.lime import TextExplainer
from typing import List
from eli5.lime.samplers import MaskingTextSampler, MaskingTextSamplers
from transformers import AutoTokenizer
from IPython.core.display import HTML, display


import torch 
device =torch.device('cuda')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))

In [8]:
tokenizer = AutoTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [10]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)

  with torch.no_grad():
      #m = model(**inputs)
      #print(model(**inputs).logits)
      logits = model(**inputs).logits
      scores = model(**inputs)[0].softmax(1).detach().cpu().numpy()

  #predicted_class_id = logits.argmax().item()
  # classifications = ['Introduction','Depression','Grey Area']
  # classification = logits.argmax().item()
  # class_name = classifications[classification]
  
  return scores


In [11]:
text = 'मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी'

In [12]:
def get_vocab(data):
  inputs = tokenizer(data,
                   return_tensors='pt',
                   padding=True,
                   truncation=True,
                   max_length=128)

  # Get the feature names for the first example
  token_ids = inputs['input_ids'][0]
  tokens = tokenizer.decode(token_ids).split()
  feature_names = tokens
  return feature_names

In [13]:
te = TextExplainer( random_state=42,sampler=MaskingTextSampler())

In [48]:
inputs = tokenizer('मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी', 
                   return_tensors='pt', 
                   padding=True, 
                   truncation=True, 
                   max_length=128)

# Get the feature names for the input
token_ids = inputs['input_ids'][0]
tokens = tokenizer.decode(token_ids).split()
feature_names = [f'token_{i}' for i in range(len(tokens))]

# Make predictions using the model
outputs = model(inputs['input_ids'], inputs['attention_mask'])
print(outputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3109, -0.2764,  0.4995]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [16]:
# Show the explanation with the actual words
te.fit('मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी', adapter)
explanation = te.show_weights(target_names=['Introduction','Depression','Grey Area'], feature_names=get_vocab(text))
print(explanation)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


ValueError: ignored

In [10]:
te.fit('मैं न्यूयॉर्क में अपने दादा-दादी के साथ पैदा हुई और पली-बढ़ी', adapter)
te.show_prediction(target_names=['Introduction','Depression','Grey Area'])

{'input_ids': tensor([[  101,   889,   884,  ...,     0,     0,     0],
        [  101,   913, 12151,  ...,     0,     0,     0],
        [  101,   913,   902,  ...,     0,     0,     0],
        ...,
        [  101,   884, 13874,  ...,     0,     0,     0],
        [  101,   889,   884,  ...,   102,     0,     0],
        [  101,   889,   890,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


In [ ]:
te.fit(docs[0], clf.predict_proba)
te.show_prediction(target_names=newsgroups_train.target_names)